# Introduction

OK we have the basics for building and running `Command`, Let's add a bit to the `Command` class so we have an `initialize()` method to setup the command, an `isFinished()` method to test whether the command is finished, an `end()` that cleans up, and a `schedule()` method that adds the command to the list of commands being run.

Once we do that we will explore 3 kinds of commands - one that runs for some number of comman cycles, one that runs for some number milliseconds, and one that lets you build a sequence of commands.

# Restart from Last Session

When we last left our intrepid programers, this was the `Command` class:

In [1]:
// This is a Command class, initially we will just start with a constructor,
// naming, and an execute method.
class Command
{
    String m_name;
    Command()
    {
        // By default the name is the class name. If this is called
        // from the constructor of an extending class, then the
        // class name will be the name of the extending class.
        m_name = getClass().getName();
        m_name = m_name.substring(m_name.lastIndexOf('$') + 1);
    }
    
    Command setName(String name)
    {
        m_name = name;
        return this;
    }
    
    String getName()
    {
        return m_name;
    }
    
    void execute()
    {
        System.out.println("executing: " + m_name);
    }
}

// create and run a command to make sure it works
Command command = new Command().setName("my new command");
command.execute();

executing: my new command


We had started an extension of this to build our own `CountedCommand` class that ran forsome number of command cycles:

In [5]:
class CountedCommand extends Command 
{
    // the step count
    long m_stepCt = 0;
    // the number of steps we run before we stop
    long m_numberOfSteps;
    
    CountedCommand(long numberOfSteps)
    {
        super();
        m_numberOfSteps = numberOfSteps;
    }
    
    @Override
    void execute()
    {
        super.execute();
        m_stepCt += 1;
        System.out.println("  stepCt: " + m_stepCt + " of " + m_numberOfSteps);
    }
}

// create and run a command to make sure it works
Command command = new CountedCommand(10).setName("test CountedCommand");
command.execute();
command.execute();
command.execute();

executing: test CountedCommand
  stepCt: 1 of 10
executing: test CountedCommand
  stepCt: 2 of 10
executing: test CountedCommand
  stepCt: 3 of 10


And this was the code that ran the command loop:

In [6]:
long RUN_TIME = 400;
long STEP_TIME = 40;

// Setup the timing for running the commands in the loop.
long startTime = System.currentTimeMillis();
System.out.println("startTime = " + startTime);
long endTime = startTime + RUN_TIME;
long nextStepEnd = startTime;

// setup the command list, and the commands to run the robot
List<Command> commandList = new ArrayList<>();
commandList.add(new Command().setName("cmd1"));
commandList.add(new Command().setName("cmd2"));
commandList.add(new CountedCommand(4));

while (System.currentTimeMillis() < endTime)
{
    // control of the step time
    long sleepTime = nextStepEnd - System.currentTimeMillis();
    if (sleepTime > 0)
    {
        Thread.sleep(sleepTime);
    }
    System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
    nextStepEnd += STEP_TIME;
    
    Iterator<Command> iter = commandList.iterator();
    while (iter.hasNext()) 
    {
        iter.next().execute();
    }
    
}

startTime = 1626803735007
time = 1626803735170; wanted: 1626803735007
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 1 of 4
time = 1626803735172; wanted: 1626803735047
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 2 of 4
time = 1626803735173; wanted: 1626803735087
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 3 of 4
time = 1626803735174; wanted: 1626803735127
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 4 of 4
time = 1626803735175; wanted: 1626803735167
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 5 of 4
time = 1626803735212; wanted: 1626803735207
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 6 of 4
time = 1626803735252; wanted: 1626803735247
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 7 of 4
time = 1626803735291; wanted: 1626803735287
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 8 of 4
time = 1626803735332; 

# Finishing the `Command` Class

What does *finishing* the `Command` class mean? Here is a partial list of the issues:
* If we attach a command to a button, there is no way to reset values to starting
  values to get ready for a second button push, so we will add an `initialize()` method
  for that. In wpilib, the `initialize()` method is called whenever a command is added
  to the command list.
* There is no way to stop a command once it is on the command list. In wpilib, at the
  beginning of each command set (or cycle) all of the commands are tested using the
  `isFinished()` method to see if they are finished. If a command is finished, the
  command is removed from the command list and the `end()` method is called so the
  command can cleanup.
* The `Command` really should not do anything unless every subclass should do it or
  it will make the subclass *work* before any methods are written.
* Since `Command` is more like a template than a real command, and it does not
  do anything - so it is probably a mistake to create a `Command` object, let's not
  make `Command` creation possible.

Here is our code for *finishing* the `Command` class:

In [2]:
// This is a Command class, initially we will just start with a constructor,
// naming, and an execute method.
abstract class Command
{
    private String m_name;
    Command()
    {
        // By default the name is the class name. If this is called
        // from the constructor of an extending class, then the
        // class name will be the name of the extending class.
        m_name = getClass().getName();
        m_name = m_name.substring(m_name.lastIndexOf('$') + 1);
    }
    
    void initialize()
    {
        
    }
    
    Command setName(String name)
    {
        m_name = name;
        return this;
    }
    
    String getName()
    {
        return m_name;
    }
    
    void execute()
    {
        //System.out.println("executing: " + m_name);
    }
    
    boolean isFinished()
    {
        return false;
    }
    
    void end(boolean interrupted)
    {
        
    }
}

// create and run a command to make sure it works - which we can't
// anymore because Commanbd is abstract
// Command command = new Command().setName("my new command");
// command.execute();

# Subclasses, Abstract Classes, Interfaces

A discussion about whether there is a shared base implementation (subclasses and abstract classes) - or - just a common way to interact with an object.

# Finishing our `CountedCommand` Class

To finish our `CountedCommand` class we need to implement the methods we just added to the `Command`.

Just as a side note, in our 2020-2021 robot the shooter used a pneumatic cylinder to lift the ball
from the collection pen into the shooter. The command for that was very much like the `CountedCommand`.
The `initialize()` set the cylinder so the lift was down; the `execute()` method set the cylinder so
the lift was up; the number of steps the cylinder was up was tuned to be the smallest number that
produced reliable delivery; and the `end()` method reset the cylinder so the lift was down and the
next ball could roll into position. So while this `CountedCommand` seems a bit like a demonstration
command, it is a good start for doing something useful.

The big things than changed between the original version and this version are:
* the step count is completely managed by the command, so we added a `private` modifier;
* The number of steps is set in the constructor, and cannot be changed, so we added a `final` modifier;
* Added an `initialize()` method that resets the step counter to `0` (note the `@Override` annotation);
* Added an `isFinished()` method that returns `false` while the step count is less than the target number
  of steps and `true` after that;
* Removed the call to the `super.execute()` method in the `execute()` (since we made `Command` abstract and
  removed functionality from any call we expect would be overridden). Added the command name to the
  print out from the `execute()`.
  
We then did a little test code that made sure the step counting and `isFinished()` worked correctly; follwed by
an initialize and a rerun of the test loop:

In [3]:
class CountedCommand extends Command 
{
    // the step count
    private long m_stepCt = 0;
    // the number of steps we run before we stop
    final long m_numberOfSteps;
    
    CountedCommand(long numberOfSteps)
    {
        super();
        m_numberOfSteps = numberOfSteps;
    }
    
    @Override
    void initialize()
    {
        m_stepCt = 0;
    }
    
    
    @Override
    void execute()
    {
        m_stepCt += 1;
        System.out.println(getName()  + ":  stepCt: " + m_stepCt + " of " + m_numberOfSteps);
    }
    @Override
    boolean isFinished()
    {
        return m_stepCt >= m_numberOfSteps;
    }
}

// create and run a command to make sure it works
Command command = new CountedCommand(2).setName("test CountedCommand");
while (!command.isFinished()) {
    command.execute();
}

System.out.println("\nReset command:");
command.initialize();
while (!command.isFinished()) {
    command.execute();
}


test CountedCommand:  stepCt: 1 of 2
test CountedCommand:  stepCt: 2 of 2

Reset command:
test CountedCommand:  stepCt: 1 of 2
test CountedCommand:  stepCt: 2 of 2


# Making the `Command` Loop a `CommandScheduler`

In wpilib the command loop is in a `CommandScheduler` that handles things like adding commands to the list, removing commands from the list either by request or when they finish, and handing subsystem dependencies so there are not multiple commands trying to run the same subsystem. Let's put a `CommandScheduler` class around the command loop
so we don't keep copying new versions of it into cells. We did this in a couple steps and I will try to
retrace that here.

The first step was wrapping the current code in a class, adding a constructor that initialized
the `RUN_TIME` and `STEP_TIME`, and putting the rest of the code in a `run()` method:

In [5]:
class CommandScheduler
{
    final long RUN_TIME;
    final long STEP_TIME;
    
    CommandScheduler(long runTime, long stepTime)
    {
        RUN_TIME = runTime;
        STEP_TIME = stepTime;
    }

    void run()
    {
        // Setup the timing for running the commands in the loop.
        long startTime = System.currentTimeMillis();
        System.out.println("startTime = " + startTime);
        long endTime = startTime + RUN_TIME;
        long nextStepEnd = startTime;

        // setup the command list, and the commands to run the robot
        List<Command> commandList = new ArrayList<>();
        commandList.add(new Command().setName("cmd1"));
        commandList.add(new Command().setName("cmd2"));
        commandList.add(new CountedCommand(4));

        while (System.currentTimeMillis() < endTime)
        {
            // control of the step time
            long sleepTime = nextStepEnd - System.currentTimeMillis();
            if (sleepTime > 0)
            {
                Thread.sleep(sleepTime);
            }
            System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
            nextStepEnd += STEP_TIME;

            Iterator<Command> iter = commandList.iterator();
            while (iter.hasNext()) 
            {
                iter.next().execute();
            }
        }
    }
}

// Create a command scheduler and run it
CommandScheduler commandScheduler = new CommandScheduler(400, 40);
commandScheduler.run();

CompilationException: 

Oh yeah, we made `Command` abstract we can't use `new Command()`, so lets use 3
different `CountedCommand` objects instead of making
two `Command` objects.

Also, once we put all this code into a `class` the rules for compiling are a
little different, and there is a thing that can happen during a `sleep()` where the program is killed
by the user, which will interrupt the sleep before it finishes. This only happens when something
is trying to stop a program. I'll talk about error handling in detail in a later talk. For now we
will just use a `try catch` wrapper for the sleep, and if we catch the `InterruptedException` we will
`break` out of the loop so the program can quit.

In [11]:
class CommandScheduler
{
    final long RUN_TIME;
    final long STEP_TIME;
    
    CommandScheduler(long runTime, long stepTime)
    {
        RUN_TIME = runTime;
        STEP_TIME = stepTime;
    }

    void run()
    {
        // Setup the timing for running the commands in the loop.
        long startTime = System.currentTimeMillis();
        System.out.println("startTime = " + startTime);
        long endTime = startTime + RUN_TIME;
        long nextStepEnd = startTime;

        // setup the command list, and the commands to run the robot
        List<Command> commandList = new ArrayList<>();
        commandList.add(new CountedCommand(10).setName("cmd1"));
        commandList.add(new CountedCommand(5).setName("cmd2"));
        commandList.add(new CountedCommand(4));

        while (System.currentTimeMillis() < endTime)
        {
            // control of the step time
            long sleepTime = nextStepEnd - System.currentTimeMillis();
            if (sleepTime > 0)
            {
                try {
                    Thread.sleep(sleepTime);
                } catch (InterruptedException e) {
                    break;
                }
            }
            System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
            nextStepEnd += STEP_TIME;

            Iterator<Command> iter = commandList.iterator();
            while (iter.hasNext()) 
            {
                iter.next().execute();
            }
        }
    }
}

// Create a command scheduler and run it
CommandScheduler commandScheduler = new CommandScheduler(400, 40);
commandScheduler.run();

startTime = 1626837645605
time = 1626837645606; wanted: 1626837645605
cmd1:  stepCt: 1 of 10
cmd2:  stepCt: 1 of 5
CountedCommand:  stepCt: 1 of 4
time = 1626837645645; wanted: 1626837645645
cmd1:  stepCt: 2 of 10
cmd2:  stepCt: 2 of 5
CountedCommand:  stepCt: 2 of 4
time = 1626837645689; wanted: 1626837645685
cmd1:  stepCt: 3 of 10
cmd2:  stepCt: 3 of 5
CountedCommand:  stepCt: 3 of 4
time = 1626837645725; wanted: 1626837645725
cmd1:  stepCt: 4 of 10
cmd2:  stepCt: 4 of 5
CountedCommand:  stepCt: 4 of 4
time = 1626837645767; wanted: 1626837645765
cmd1:  stepCt: 5 of 10
cmd2:  stepCt: 5 of 5
CountedCommand:  stepCt: 5 of 4
time = 1626837645806; wanted: 1626837645805
cmd1:  stepCt: 6 of 10
cmd2:  stepCt: 6 of 5
CountedCommand:  stepCt: 6 of 4
time = 1626837645849; wanted: 1626837645845
cmd1:  stepCt: 7 of 10
cmd2:  stepCt: 7 of 5
CountedCommand:  stepCt: 7 of 4
time = 1626837645887; wanted: 1626837645885
cmd1:  stepCt: 8 of 10
cmd2:  stepCt: 8 of 5
CountedCommand:  stepCt: 8 of 4
time =

Good progress - but, the `CountedCommand` never stops,. Let's add the check for commands finishing:

In [16]:
class CommandScheduler
{
    final long RUN_TIME;
    final long STEP_TIME;
    
    CommandScheduler(long runTime, long stepTime)
    {
        RUN_TIME = runTime;
        STEP_TIME = stepTime;
    }

    void run()
    {
        // Setup the timing for running the commands in the loop.
        long startTime = System.currentTimeMillis();
        System.out.println("startTime = " + startTime);
        long endTime = startTime + RUN_TIME;
        long nextStepEnd = startTime;

        // setup the command list, and the commands to run the robot
        List<Command> commandList = new ArrayList<>();
        commandList.add(new CountedCommand(10).setName("cmd1"));
        commandList.add(new CountedCommand(5).setName("cmd2"));
        commandList.add(new CountedCommand(4));

        while (System.currentTimeMillis() < endTime)
        {
            // control of the step time
            long sleepTime = nextStepEnd - System.currentTimeMillis();
            if (sleepTime > 0)
            {
                try {
                    Thread.sleep(sleepTime);
                } catch (InterruptedException e) {
                    break;
                }
            }
            System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
            nextStepEnd += STEP_TIME;
            
            // check the currently scheduled commands and remove any that have finished
            Iterator<Command> iter = commandList.iterator();
            while (iter.hasNext()) {
                Command command = iter.next();
                if (command.isFinished()) {
                    // remove the command we just got with the call to next()
                    iter.remove();
                    // call the end() method so the command can finish, the argument is false
                    // because the command was not interupted, but ended as expected.
                    command.end(false);
                }
            }

            // run the scheduled commands
            iter = commandList.iterator();
            while (iter.hasNext()) 
            {
                iter.next().execute();
            }
        }
    }
}

// Create a command scheduler and run it
CommandScheduler commandScheduler = new CommandScheduler(400, 40);
commandScheduler.run();

startTime = 1626843781146
time = 1626843781146; wanted: 1626843781146
cmd1:  stepCt: 1 of 10
cmd2:  stepCt: 1 of 5
CountedCommand:  stepCt: 1 of 4
time = 1626843781186; wanted: 1626843781186
cmd1:  stepCt: 2 of 10
cmd2:  stepCt: 2 of 5
CountedCommand:  stepCt: 2 of 4
time = 1626843781227; wanted: 1626843781226
cmd1:  stepCt: 3 of 10
cmd2:  stepCt: 3 of 5
CountedCommand:  stepCt: 3 of 4
time = 1626843781267; wanted: 1626843781266
cmd1:  stepCt: 4 of 10
cmd2:  stepCt: 4 of 5
CountedCommand:  stepCt: 4 of 4
time = 1626843781306; wanted: 1626843781306
cmd1:  stepCt: 5 of 10
cmd2:  stepCt: 5 of 5
time = 1626843781347; wanted: 1626843781346
cmd1:  stepCt: 6 of 10
time = 1626843781386; wanted: 1626843781386
cmd1:  stepCt: 7 of 10
time = 1626843781431; wanted: 1626843781426
cmd1:  stepCt: 8 of 10
time = 1626843781466; wanted: 1626843781466
cmd1:  stepCt: 9 of 10
time = 1626843781510; wanted: 1626843781506
cmd1:  stepCt: 10 of 10
time = 1626843781548; wanted: 1626843781546


Now this looks really good, all our commands ended on the step we expected then to end.

The only remaining issue is that we are putting commands on the `commandList` in the run method, and the
run method should only run scheduled commands. We should be asking the `CommandScheduler` to `schedule`
commands. in the wpilib framework, we attach commands to buttons on the control station and when the
button is pushed, the command is scheduled. When we first create the robot we specify commands that should
be on the command list.

After consulting what really happens in wpilib, we can implement a really good approximation, which is
that we add a `schedule()` method that adds a command to a `m_toBeScheduled` list and at the start of
every command cycle we add everything on that list to the command list:

While we are doing that, we remove all the command creation from the `run()` method, and put it after creating `CommandScheduler` object.


In [18]:
class CommandScheduler
{
    final long RUN_TIME;
    final long STEP_TIME;
    
    // this is the list of commands to be executed the next time commands are run.
    private final List<Command> m_commandList = new ArrayList<>();
    // this is the list of commans waiting to be scheduled the next time commands are run.
    private final List<Command> m_toBeScheduled = new ArrayList<>();

    CommandScheduler(long runTime, long stepTime)
    {
        RUN_TIME = runTime;
        STEP_TIME = stepTime;
    }
    
    void schedule(Command command)
    {
        m_toBeScheduled.add(command);
    }
    
    void run()
    {
        // Setup the timing for running the commands in the loop.
        long startTime = System.currentTimeMillis();
        System.out.println("startTime = " + startTime);
        long endTime = startTime + RUN_TIME;
        long nextStepEnd = startTime;

        // Initialize all the commands
        Iterator<Command> iter = m_toBeScheduled.iterator();
        while (iter.hasNext()) {
        // get the next command to be scheduled
            Command command = iter.next();
            // remove it from the m_toBeScheduled list
            iter.remove();
            // initialize it
            command.initialize();
            // and add it to the scheduled command list
            m_commandList.add(command);
        }

        while (System.currentTimeMillis() < endTime)
        {
            // control of the step time
            long sleepTime = nextStepEnd - System.currentTimeMillis();
            if (sleepTime > 0)
            {
                try {
                    Thread.sleep(sleepTime);
                } catch(InterruptedException e) {
                    break;
                }
            }
            System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
            nextStepEnd += STEP_TIME;
            
            // check the currently scheduled commands and remove any that have finished
            iter = m_commandList.iterator();
            while (iter.hasNext()) {
                Command command = iter.next();
                if (command.isFinished()) {
                    // remove the command we just got with the call to next()
                    iter.remove();
                    // call the end() method so the command can finish, the argument is false
                    // because the command was not interupted, but ended as expected.
                    command.end(false);
                }
            }

            // run the commands
            iter = m_commandList.iterator();
            while (iter.hasNext()) 
            {
                iter.next().execute();
            }

        }
    }
}

CommandScheduler commandScheduler = new CommandScheduler(400, 40);
Command countedCommand4 = new CountedCommand(4);
commandScheduler.schedule(new CountedCommand(10).setName("cmd1"));
commandScheduler.schedule(new CountedCommand(5).setName("cmd2"));
commandScheduler.schedule(countedCommand4);
commandScheduler.run();


System.out.println("\nReschedule countedCommand4 and run the command scheduler");
commandScheduler.schedule(countedCommand4);
commandScheduler.run();

startTime = 1626844493066
time = 1626844493066; wanted: 1626844493066
cmd1:  stepCt: 1 of 10
cmd2:  stepCt: 1 of 5
CountedCommand:  stepCt: 1 of 4
time = 1626844493107; wanted: 1626844493106
cmd1:  stepCt: 2 of 10
cmd2:  stepCt: 2 of 5
CountedCommand:  stepCt: 2 of 4
time = 1626844493146; wanted: 1626844493146
cmd1:  stepCt: 3 of 10
cmd2:  stepCt: 3 of 5
CountedCommand:  stepCt: 3 of 4
time = 1626844493186; wanted: 1626844493186
cmd1:  stepCt: 4 of 10
cmd2:  stepCt: 4 of 5
CountedCommand:  stepCt: 4 of 4
time = 1626844493230; wanted: 1626844493226
cmd1:  stepCt: 5 of 10
cmd2:  stepCt: 5 of 5
time = 1626844493266; wanted: 1626844493266
cmd1:  stepCt: 6 of 10
time = 1626844493309; wanted: 1626844493306
cmd1:  stepCt: 7 of 10
time = 1626844493346; wanted: 1626844493346
cmd1:  stepCt: 8 of 10
time = 1626844493386; wanted: 1626844493386
cmd1:  stepCt: 9 of 10
time = 1626844493430; wanted: 1626844493426
cmd1:  stepCt: 10 of 10
time = 1626844493466; wanted: 1626844493466

Reschedule countedCo

OK, good session! We understand most of the details of command based programming and how it works
bucause we've just written the core components, `Command` and `CommandScheduler`, of the wpilib
framework. In our next section we will build and test a few commands. Then we will talk a bit about
bad things you can do in commands that really break how this approach works.

And if we have time, we will start to move to the **Idea** java development environment - which has a
lot of cool tools and capabilities we have not seen when using notebooks.